### Importing Data and Libraries

In [1]:
# load some default Python modules
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("Your Path", nrows = 10_000_000)
print("shape of train data", train.shape)
train.head()

shape of train data (10000000, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
# datatypes
train.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [4]:
# Basic Stats of the data set
train.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.000000e+07,1.000000e+07,1.000000e+07,9.999931e+06,9.999931e+06,1.000000e+07
mean,1.133854e+01,-7.250775e+01,3.991934e+01,-7.250897e+01,3.991913e+01,1.684793e+00
std,9.799930e+00,1.299421e+01,9.322539e+00,1.287532e+01,9.237280e+00,1.323423e+00
min,-1.077500e+02,-3.439245e+03,-3.492264e+03,-3.426601e+03,-3.488080e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073491e+01,-7.399139e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075263e+01,-7.398016e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396710e+01,4.076712e+01,-7.396367e+01,4.076810e+01,2.000000e+00
max,1.273310e+03,3.457626e+03,3.344459e+03,3.457622e+03,3.351403e+03,2.080000e+02


**following thing that i can notice**
- Fare amount is negative and it doesn't make sense
- long and lat also looks off
- maximum passanger count is 208 which looks odd


In [ ]:
# there are 211 records where fare amount is negaive 
train[train.fare_amount <0].shape

In [ ]:
# delete negative fare record
print("old size: %d" % len(train))
train = train[train.fare_amount >=0]
print("New size: %d" % len(train))

In [ ]:
# check missing data
train.isnull().sum()

**There are 69 records where longitud and latitude are missing, we will drop them from the data**

In [ ]:
print("old size: %d" % len(train))
train = train.dropna(how='any', axis=0)
print("New size after dropping missing value: %d" % len(train))

In [ ]:
# Lets see the distribution of fare amount 
train.fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")

In [ ]:
# logrithmic distribution of the fare amount
np.log1p(train.fare_amount).hist(bins=100, figsize = (16,8))

- Looks like the distribution is highly skewed and frequency above 100 is very less
- we will plot below 100 and above 100 separately 

In [ ]:
# Lets see the distribution of fare amount less than 100
train[train.fare_amount <100 ].fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")

- There are few points between 40 and 60 dollars which has slightly high frequency and that could be airport trips

In [ ]:
train[train.fare_amount >100 ].shape

- we can see here that there are total 1977 trips which are above 100 dollars
- some of them might be outliers or few of them might be long distance trip from/to airport, we will see it in later sectionn

In [ ]:
# Lets see the distribution of fare amount more than 100
train[train.fare_amount >100 ].fare_amount.hist(bins=100, figsize = (16,8))
plt.xlabel("Fare Amount")
plt.ylabel("Frequency")

#### Lets also check passanger count distribution

In [ ]:
# checking for passanger count greater than 7
train[train.passenger_count >7].passenger_count.hist(bins=10, figsize = (16,8))
plt.xlabel("Passanger Count")
plt.ylabel("Frequency")

There is only one entry for 208 passanger count, we will drop this record in data cleaning section

In [ ]:
# checking for passanger count less than 7
train[train.passenger_count <7].passenger_count.hist(bins=10, figsize = (16,8))
plt.xlabel("Passanger Count")
plt.ylabel("Frequency")

- Most of the trips are taken by single passanger
- we will try to see if there is any relation between passanger count and fare amout

In [ ]:
plt.figure(figsize= (16,8))
sns.boxplot(x = train[train.passenger_count< 7].passenger_count, y = train.fare_amount)

- As we can see from the box plot median price of each passanger counts looks similar
- we will try to see if there is any relationship between passanger count and fare amount using correlation factor

In [ ]:
train[train.passenger_count <10][['fare_amount','passenger_count']].corr()

**There is very weak correlation (0.013) between fare amount and passanger count**, 

got to know from online sources that there is no additional fare for extra passanger which means it doesn't have any effect on the fare, for now we can drop the passanger count columns

In [ ]:
# dropping passanger count column
train = train.drop('passenger_count',1)
test = test.drop('passenger_count',1)

### Lets read the test data

In [ ]:
test = pd.read_csv("Your Path")
print("shape of test data", test.shape)
test.head()

In [ ]:
#check for missing value
test.isnull().sum()

There are no missing value in test data set

In [ ]:
# checking for basic stats
test.describe()

We will store the minimum and maximum of the longitude and latitude from test data set and filter the train data set for those data points

In [ ]:
min(test.pickup_longitude.min(),test.dropoff_longitude.min()), \
max(test.pickup_longitude.max(),test.dropoff_longitude.max())

In [ ]:
min(test.pickup_latitude.min(),test.dropoff_latitude.min()), \
max(test.pickup_latitude.max(),test.dropoff_latitude.max())

In [ ]:
# this function will also be used with the test set below
def select_within_test_boundary(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])

In [ ]:
BB = (-74.5, -72.8, 40.5, 41.8)
print('Old size: %d' % len(train))
train = train[select_within_test_boundary(train, BB)]
print('New size: %d' % len(train))

- Now we have sliced the train data records as per the coordinates of the test data

### Manual Feature Engineering
- Adding distance metrics
- few time based variables

In [ ]:
def prepare_time_features(df):
    df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    df['hour_of_day'] = df.pickup_datetime.dt.hour
#     df['week'] = df.pickup_datetime.dt.week
    df['month'] = df.pickup_datetime.dt.month
    df["year"] = df.pickup_datetime.dt.year
#     df['day_of_year'] = df.pickup_datetime.dt.dayofyear
#     df['week_of_year'] = df.pickup_datetime.dt.weekofyear
    df["weekday"] = df.pickup_datetime.dt.weekday
#     df["quarter"] = df.pickup_datetime.dt.quarter
    df["day_of_month"] = df.pickup_datetime.dt.day
    
    return df

In [ ]:
train = prepare_time_features(train)
test = prepare_time_features(test)

In [ ]:
test.head()

In [ ]:
# time frame plot chart from https://www.kaggle.com/nicapotato/taxi-rides-time-analysis-and-oof-lgbm
def time_slicer(df, timeframes, value, color="purple"):
    """
    Function to count observation occurrence through different lenses of time.
    """
    f, ax = plt.subplots(len(timeframes), figsize = [12,12])
    for i,x in enumerate(timeframes):
        df.loc[:,[x,value]].groupby([x]).mean().plot(ax=ax[i],color=color)
        ax[i].set_ylabel(value.replace("_", " ").title())
        ax[i].set_title("{} by {}".format(value.replace("_", " ").title(), x.replace("_", " ").title()))
        ax[i].set_xlabel("")
    ax[len(timeframes)-1].set_xlabel("Time Frame")
    plt.tight_layout(pad=0)

In [ ]:
# calculate-distance-between-two-latitude-longitude-points-haversine-formula 
# Returns distance in miles
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))   # 2*R*asin...

In [ ]:
train['distance_miles'] = distance(train.pickup_latitude, train.pickup_longitude, \
                                      train.dropoff_latitude, train.dropoff_longitude)

In [ ]:
test['distance_miles'] = distance(test.pickup_latitude, test.pickup_longitude, \
                                      test.dropoff_latitude, test.dropoff_longitude)

#### Calculating pickup and drop distance from all 3 airports of Air Ports

In [ ]:
def transform(data):
    # Distances to nearby airports, 
    jfk = (-73.7781, 40.6413)
    ewr = (-74.1745, 40.6895)
    lgr = (-73.8740, 40.7769)
    nyc = (-74.0064, 40.7142)


    data['distance_to_center'] = distance(nyc[1], nyc[0],
                                      data['pickup_latitude'], data['pickup_longitude'])
    data['pickup_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = distance(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = distance(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data

train = transform(train)
test = transform(test)

In [ ]:
time_slicer(df=train, timeframes=['year', "month", "day_of_month", "weekday", "hour_of_day"],
            value = "fare_amount", color="blue")

In [ ]:
time_slicer(df=train, timeframes=['year', "month", "day_of_month", "weekday", "hour_of_day"], 
            value = "distance_miles", color = "green")

- Delete these 15 record where distance covered and fare amount are zero, which won't help our model

In [ ]:
train[(train['distance_miles']==0)&(train['fare_amount']==0)]

In [ ]:
print("old size: %d" % len(train))
train = train.drop(index= train[(train['distance_miles']==0)&(train['fare_amount']==0)].index, axis=0)
print("New size: %d" % len(train))

- There are 92 records where fare amount is zero but distance travelled is greater than 0, we will drop such case

In [ ]:
train[train['fare_amount']==0].shape

In [ ]:
train[train['fare_amount']==0].index

In [ ]:
print("old size: %d" % len(train))
train = train.drop(index= train[train['fare_amount']==0].index, axis=0)
print("New size: %d" % len(train))

- There are 53 record where fare amount is less than 2.5 dollars (0.01 dollars) which doesn't make sense as the base fare for any taxi in new york is 2.5 dollars, we will drop those cases

In [ ]:
print("old size: %d" % len(train))
train = train.drop(index= train[train['fare_amount'] <= 2.49].index, axis=0)
print("New size: %d" % len(train))

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
print("old size: %d" % len(train))
train = train.drop(index= train[(train.distance_miles ==0)&(train.fare_amount >3.5)].index, axis = 0)
print("New size: %d" % len(train))

In [ ]:
print("old size: %d" % len(train))
train = train.drop(index = train[train.fare_amount> 200].index, axis = 0)
print("New size: %d" % len(train))

In [ ]:
sns.jointplot(train.distance_miles, train.fare_amount, size= 12, ratio= 1)

In [ ]:
print("old size: %d" % len(train))
train = train.drop(index =train[(train.distance_miles < 2)&(train.fare_amount >75)].index, axis=0)
print("New size: %d" % len(train))

In [ ]:
# print("old size: %d" % len(train))
# train = train.drop(index = train[(train.distance_miles > 42)].index, axis = 0)
# print("New size: %d" % len(train))

In [ ]:
# create copy of the data set
df_train = train.drop(columns= ['key','pickup_datetime'], axis= 1).copy()
df_test = test.drop(columns= ['key','pickup_datetime'], axis= 1).copy()
print(df_train.shape)
print(df_test.shape)

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('fare_amount', axis=1),
                                                    df_train['fare_amount'], test_size=0.15, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Modelling

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

### Linear Baseline

In [ ]:
#Validation function
n_folds = 5

def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, df_train.drop('fare_amount', axis=1).values, 
                                   df_train['fare_amount'], scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
linear = make_pipeline(RobustScaler(), LinearRegression())
score = rmse_cv(linear)
print("\nLinear score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
linear.fit(X_train,y_train)
y_lin_pred = linear.predict(X_test)
print("test score", rmse(y_test,y_lin_pred))
prediction = linear.predict(df_test).tolist()
test1 = pd.read_csv("Your Path")
holdout = pd.DataFrame({'key': test1['key'], 'fare_amount': prediction})
holdout.to_csv("Your Path", index=False)
holdout.head()

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmse_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmse_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

### Baseline XGB

In [ ]:
params = {
   
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

In [ ]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params)

In [ ]:
prediction = model.predict(xgb.DMatrix(df_test), ntree_limit = model.best_ntree_limit).tolist()
test1 = pd.read_csv("Your Path")
holdout = pd.DataFrame({'key': test1['key'], 'fare_amount': prediction})
holdout.to_csv("Your Path", index=False)
holdout.head()

#### Baseline LGBM

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'max_depth': 7,
    'verbose': 0
}

In [ ]:
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=999,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

In [ ]:
prediction = gbm.predict(df_test,num_iteration= gbm.best_iteration).tolist()
test1 = pd.read_csv("Your Path")
holdout = pd.DataFrame({'key': test1['key'], 'fare_amount': prediction})
holdout.to_csv("Your Path", index=False)
holdout.head()

### Ensemble -- Weighted Average of XGB and LGB

In [ ]:
lgb_baseline = gbm.predict(df_test,num_iteration= gbm.best_iteration)
xgb_baseline = model.predict(xgb.DMatrix(df_test), ntree_limit = model.best_ntree_limit)

In [ ]:
ensemble = lgb_baseline* 0.25 + xgb_baseline*0.75
test1 = pd.read_csv("Your Path")
holdout = pd.DataFrame({'key': test1['key'], 'fare_amount': ensemble})
holdout.to_csv("Your Path", index=False)
holdout.head()